In [4]:
import requests
import pprint
import pandas as pd

from bs4 import BeautifulSoup
from io import BytesIO

In [8]:
JPX_SHORT_SELLING_URL = "https://www.jpx.co.jp/english/markets/public/short-selling/index.html"
HISTORICAL_SHORT_SELLING_URL = "https://www.jpx.co.jp/english/markets/public/short-selling/00-archives-{index}.html"

urls = [JPX_SHORT_SELLING_URL]

# Collect trailing historical 12 month url links
for index in range(1, 13):
    urls.append(HISTORICAL_SHORT_SELLING_URL.format(index=str(index).zfill(2)))
pprint.pprint(urls)

# Spoof web crawler User Agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

"""
1. Crawl JPX urls
2. Find the short interest spread sheet
3. Read xls file + Load raw data into DataFrame
"""
partition_dfs = []
for url in urls:
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
    
        xls_refs = soup.find_all('a', href=lambda href: href and 'Short_Positions.xls' in href)
        xls_urls = [ref['href'] for ref in xls_refs]

        for xls_url in xls_urls:
            excel_url = f"https://www.jpx.co.jp{xls_url}"
            excel_response = requests.get(excel_url, headers=headers)
            if excel_response.status_code == 200:
                excel_data = BytesIO(excel_response.content)
                df = pd.read_excel(excel_data, engine='xlrd')
                partition_dfs.append(df)
    print(f'Finished processing {url}')

# Snake case mapping between JPX spreadsheet schema for easier queries
schema = [
    "date_of_calculation",
    "code_of_stock",
    "name_of_stock",
    "name_of_short_seller",
    "address_of_short_seller",
    "name_of_discretionary_investment_contractor",
    "address_of_discretionary_investment_contractor",
    "name_of_investment_fund",
    "ratio_of_short_positions_to_shares_outstanding",
    "number_of_short_positions_in_shares",
    "number_of_short_positions_in_trading_units",
    "date_of_calculation_in_previous_reporting",
    "ratio_of_short_positions_in_previous_reporting",
    "notes"
]

"""
1. Sanitize data
    i.  Only process 3350 rows
    ii. Filter out Name of Stock (Japanese) column
2. Merge partitioned DataFrames
"""
cleaned_rows = []
for partition_df in partition_dfs:
    data_rows = partition_df.values.tolist()

    for row in data_rows:
        cleaned_row = row[1:]
        # Filter only for metaplanet stock
        if cleaned_row[1] == 3350:
            # Filter out Name of Stock (Japanese) name
            cleaned_rows.append(cleaned_row[:2] + cleaned_row[3:])
### Debugging purposes
# print(schema)
# print(cleaned_data)

# Write historical bootstrapped short DataFrame into a local .csv file
file_name = 'short_position_3350_bootstrap_data.csv'
date_column = 'date_of_calculation'

df = pd.DataFrame(cleaned_rows, columns=schema)
df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
df = df.sort_values(by=date_column, ascending=False)
df.to_csv(file_name, index=False)

print(f"Data saved to {file_name}.")

['https://www.jpx.co.jp/english/markets/public/short-selling/index.html',
 'https://www.jpx.co.jp/english/markets/public/short-selling/00-archives-01.html',
 'https://www.jpx.co.jp/english/markets/public/short-selling/00-archives-02.html',
 'https://www.jpx.co.jp/english/markets/public/short-selling/00-archives-03.html',
 'https://www.jpx.co.jp/english/markets/public/short-selling/00-archives-04.html',
 'https://www.jpx.co.jp/english/markets/public/short-selling/00-archives-05.html',
 'https://www.jpx.co.jp/english/markets/public/short-selling/00-archives-06.html',
 'https://www.jpx.co.jp/english/markets/public/short-selling/00-archives-07.html',
 'https://www.jpx.co.jp/english/markets/public/short-selling/00-archives-08.html',
 'https://www.jpx.co.jp/english/markets/public/short-selling/00-archives-09.html',
 'https://www.jpx.co.jp/english/markets/public/short-selling/00-archives-10.html',
 'https://www.jpx.co.jp/english/markets/public/short-selling/00-archives-11.html',
 'https://www

KeyError: 'date_of_calculation'